In [13]:
import pandas as pd
import numpy as np
import sklearn as sklearn
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import scipy.stats as stats

from utils.base_set import X_train, y_train, seed

### Decision Tree

In [14]:
model = DecisionTreeClassifier(random_state=seed)
p = X_train.shape[1]
n = X_train.shape[0]
k = 5
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
params = {
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": stats.randint(3, np.floor(np.sqrt(n))),
    "max_features": stats.randint(1, p),
    
}
n_iter = 100
classifier_Dtree = RandomizedSearchCV(estimator=model, 
                                    param_distributions=params, 
                                    n_iter=n_iter, 
                                    cv=cv, 
                                    scoring='roc_auc',
                                    random_state=seed)
classifier_Dtree.fit(X_train, y_train)

21.18962010041709


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=8241, shuffle=True),
                   estimator=DecisionTreeClassifier(random_state=8241),
                   n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x761aa7cee200>,
                                        'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x761aa7fc2bf0>},
                   random_state=8241, scoring='roc_auc')

In [17]:
best = classifier_Dtree.best_params_
auc_roc = classifier_Dtree.best_score_
print(best)
print(auc_roc)

{'criterion': 'entropy', 'max_depth': 3, 'max_features': 135}
0.6842775475222275


In [18]:
columns_to_keep = [f"param_{param}" for param in params.keys()] + ['mean_test_score','rank_test_score']
pd.DataFrame(classifier_Dtree.cv_results_).sort_values("rank_test_score")[columns_to_keep]

,param_criterion,param_max_depth,param_max_features,mean_test_score,rank_test_score
46,entropy,3,135,0.684278,1
55,entropy,3,14,0.659835,2
27,log_loss,4,148,0.651703,3
85,log_loss,3,77,0.651348,4
60,entropy,9,91,0.646844,5
...,...,...,...,...,...
94,entropy,17,165,0.534934,96
65,log_loss,11,21,0.533663,97
17,gini,7,24,0.528307,98
12,log_loss,3,17,0.523766,99
